In [1]:
# read in the module

import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

import numpy as np
import geopandas as gpd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn

from scipy import stats
from scipy.special import boxcox1p

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling
%matplotlib inline

# libraries for cluster
import matplotlib.pyplot as plt # For plotting
import numpy as np              # For working with numerical data
import sklearn.cluster as sklc  # For clustering
import sklearn.metrics as sklm  # For the silhouette score


### Cluster analysis

#### Read in the ULEV data

read in the raw data

In [2]:
ulev_licensed_local_authority_raw = pd.read_excel (r'data.xlsx', sheet_name='ULEV_licensed_veh0132a')

In [3]:
ulev_licensed_local_authority_raw 

,ONS LA Code (Apr-2019),Region/Local Authority,2021 Q2,2021 Q1,2020 Q4,2020 Q3,2020 Q2,2020 Q1,2019 Q4,2019 Q3,...,2014 Q1,2013 Q4,2013 Q3,2013 Q2,2013 Q1,2012 Q4,2012 Q3,2012 Q2,2012 Q1,2011 Q4
0,K02000001,United Kingdom,564694,488079,431661,373225,317268,300016,269376,245138,...,16675,14967,14034,13420,12583,12064,11315,10856,10270,9954
1,K03000001,Great Britain,558022,482403,426777,368977,313554,296466,266194,242121,...,16390,14719,13814,13222,12418,11909,11177,10745,10178,9867
2,E92000001,England,505077,435982,385981,333964,283590,270241,240243,218330,...,14549,13099,12268,11653,11017,10523,10021,9554,9142,9011
3,E12000001,North East,8198,6941,6124,5555,4793,4694,4229,4032,...,524,454,468,445,427,407,390,387,344,349
4,E06000047,County Durham,1619,1344,1224,1113,962,937,813,770,...,87,79,72,57,48,45,43,43,41,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,N09000009,Mid Ulster,244,206,187,171,165,165,144,136,...,12,11,10,9,9,8,7,7,7,7
463,N09000010,"Newry, Mourne and Down",319,272,239,206,180,175,155,155,...,24,19,17,13,7,7,7,6,6,5
464,NaN,Local Authority unknown,344,270,215,146,118,98,86,67,...,c,c,c,c,c,c,0,0,0,0
465,NaN,Vehicle under disposal,7785,6963,6464,5600,5087,2010,4566,4254,...,511,468,467,569,499,540,347,451,371,244


change the column names

In [4]:
ulev_licensed_local_authority_raw.rename(columns={"ONS LA Code (Apr-2019)":"LA_code","Region/Local Authority":"Local_Authority"},inplace=True)

Select specific columns

In [5]:
ulev_licensed_local_authority = ulev_licensed_local_authority_raw[['LA_code','Local_Authority','2019 Q4']]

In [6]:
ulev_licensed_local_authority.shape

(467, 3)

here there are 467 rows

In [7]:
ulev_licensed_local_authority.isna().sum()

LA_code            36
Local_Authority     0
2019 Q4             0
dtype: int64

drop the nas

In [8]:
ulev_licensed_local_authority = ulev_licensed_local_authority.drop(ulev_licensed_local_authority[ulev_licensed_local_authority.LA_code.isna()].index.values, axis=0)

In [9]:
ulev_licensed_local_authority.shape

(431, 3)

Read in the geographic data

In [10]:
local_authority = gpd.read_file('LAD_MAY_2021_UK_BFE_V2.shp')

DriverError: LAD_MAY_2021_UK_BFE_V2.shp: No such file or directory

In [ ]:
#region = gpd.read_file('Regions_(December_2017)_Boundaries.shp')

In [ ]:
region1 = gpd.read_file('NUTS_Level_1_(January_2018)_Boundaries.shp')

Show the region level and local authority level data

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,12))

local_authority.plot(ax=ax,label='local_authority',legend=True)
region1.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1,label='region',legend=True)
ax.set_title('Countryside listings distribution', fontdict={'fontsize':'20', 'fontweight':'3'})  #provide a title

ax.axis('off')
plt.legend()
ax.legend(fontsize = 13)
plt.show()

#### Read in charging device data

In [ ]:
charging_device_local_authority = pd.read_excel (r'data.xlsx', sheet_name='charging_device_Oct19_EVCD_01a')

In [ ]:
charging_device_local_authority

In [ ]:
charging_device_local_authority.rename(columns={"LA / Region Code":"LA_code","Local Authority / Region Name":"Local_Authority"},inplace=True)

In [ ]:
charging_device_local_authority.shape

In [ ]:
charging_device_local_authority.isna().sum()


In [ ]:
ulev_licensed_local_authority.isna().sum()

#### Merge ULEV data and the charging device data

In [ ]:
ulev_cd_df = pd.merge(ulev_licensed_local_authority,charging_device_local_authority,on='LA_code',how='outer')

In [ ]:
ulev_cd_df.isna().sum()

In [ ]:
ulev_cd_df.info()

check all the rows containing na

In [ ]:
na_rows = ulev_cd_df[ulev_cd_df['Total devices'].isna() | ulev_cd_df['2019 Q4'].isna()]

In [ ]:
na_rows 

drop the nas

In [ ]:
ulev_cd_df = ulev_cd_df.drop(na_rows.index.values,axis=0)

In [ ]:
ulev_cd_df 

#### cluster analysis (k means)

merge the ulev_cd_df with the local_authority data

In [ ]:
ulev_cd_df_local_authority = pd.merge(ulev_cd_df,local_authority,how='outer',left_on='LA_code', right_on='LAD21CD')

In [ ]:
ulev_cd_df_local_authority.info()

Check NAs

In [ ]:
ulev_cd_df_local_authority_nas = ulev_cd_df_local_authority[ulev_cd_df_local_authority['OBJECTID'].isna() | ulev_cd_df_local_authority['2019 Q4'].isna()]

In [ ]:
ulev_cd_df_local_authority_nas

In [ ]:
ulev_cd_df_local_authority = ulev_cd_df_local_authority.drop(ulev_cd_df_local_authority_nas.index.values,axis=0)

In [ ]:
ulev_cd_df_local_authority.info()

Change the data type

In [ ]:
ulev_cd_df_local_authority["Total devices"] = ulev_cd_df_local_authority["Total devices"].astype('int')
ulev_cd_df_local_authority["per 100,000 population"] = ulev_cd_df_local_authority["per 100,000 population"].astype('float')
ulev_cd_df_local_authority["2019 Q4"] = ulev_cd_df_local_authority["2019 Q4"].astype('int')

In [ ]:
ulev_cd_df_local_authority.info()

In [ ]:
ulev_cd_df_local_authority1 = ulev_cd_df_local_authority

In [ ]:
# convert df to gdf
ulev_cd_local_authority1_gdf = gpd.GeoDataFrame(ulev_cd_df_local_authority1, geometry=ulev_cd_df_local_authority1.geometry,crs='epsg:27700')


In [ ]:
ulev_cd_df_local_authority1.shape

In [ ]:
ulev_cd_local_authority1_gdf.plot()

Show the gdf data (there are some NAs so the map is not complete)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,9))

ulev_cd_local_authority1_gdf.plot(ax=ax,column='per 100,000 population',cmap='coolwarm',legend=True)
#region1.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1,label='region',legend=True)
#ax.set_title('Countryside listings distribution', fontdict={'fontsize':'20', 'fontweight':'3'})  #provide a title

#ax.axis('off')
#local_authority.plot(ax=ax,label='local_authority',edgecolor='black',facecolor='none',legend=True)
#region1.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=0.5,label='region',legend=True)

In [ ]:
data_cluster = ulev_cd_local_authority1_gdf

In [ ]:
data_cluster['population_100000'] = data_cluster['Total devices']/data_cluster['per 100,000 population']

In [ ]:
data_cluster['ULEV_per_100000_population'] = data_cluster['2019 Q4']/data_cluster['population_100000']

In [ ]:
data_cluster = data_cluster.rename(columns={'per 100,000 population':'charging_devices_per_100000_population'})

In [ ]:
data_cluster[data_cluster['charging_devices_per_100000_population']==0]

check the distribution of data(need to contain all 3 columns)

In [ ]:
fig1, axes1 = plt.subplots(1,2,figsize=(12,5))
axes1[0].hist(data_cluster['ULEV_per_100000_population'],bins=50,color='xkcd:salmon',edgecolor="white")
axes1[0].set(xlabel='ULEV_per_100000_population')
axes1[1].hist(data_cluster['charging_devices_per_100000_population'],bins=50,color='xkcd:salmon',edgecolor="white")
axes1[1].set(xlabel='charging_devices_per_100000_population')
fig1.suptitle('Distribution',fontsize=16)
fig1.tight_layout(pad=1)

In [ ]:
cols_cluster = ['ULEV_per_100000_population','charging_devices_per_100000_population']

Scale the two columns to normalise them

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scalers = [MinMaxScaler().fit(data_cluster[x].values.reshape(-1,1)) for x in cols_cluster]

In [ ]:
scalers 

In [ ]:
data_cluster_normalised = data_cluster[cols_cluster+['LA_code','Local_Authority_x','geometry']].copy()

for i in range(0, len(cols_cluster)):
    # Ditto this -- can you explain what this code is doing
    data_cluster_normalised[cols_cluster[i]] = scalers[i].transform(data_cluster_normalised[cols_cluster[i]].values.reshape(-1,1))

In [ ]:
data_cluster_normalised[data_cluster_normalised['ULEV_per_100000_population'].isna()]

In [ ]:
 data_cluster_normalised = data_cluster_normalised.drop(26)

In [ ]:
data_cluster_normalised = data_cluster_normalised.drop(330)

Here do the distribution plot for the data after normalisation

In [ ]:
plt.hist(data_cluster_normalised['ULEV_per_100000_population'],bins=50)

In [ ]:
plt.hist(data_cluster_normalised['charging_devices_per_100000_population'],bins=50)

Do the K means cluster

In [ ]:
kmeans_data = data_cluster_normalised[['charging_devices_per_100000_population','ULEV_per_100000_population','LA_code']].set_index('LA_code')

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, OPTICS

In [ ]:
c_nm   = 'KMeans' # Clustering name
k_pref = 5 # Number of clusters

kmeans = KMeans(n_clusters=k_pref, n_init=25, random_state=42).fit(kmeans_data) # The process

print(kmeans.labels_) # The results

In [ ]:
len(kmeans.labels_)

In [ ]:
Kmeansdata_with_clusters = np.hstack((kmeans_data,np.array([kmeans.labels_]).T))

In [ ]:
Kmeansdata_with_clusters

In [ ]:
data_by_cluster = []

for i in range(k_pref):
    
    this_data = []
    
    for row in Kmeansdata_with_clusters:
        
        if row[-1] == i:
            this_data.append(row)
    
    this_data = np.array(this_data)
    
    data_by_cluster.append(this_data)
    
#Which gives the following:
data_by_cluster

In [ ]:
# FIGURE PARAMETERS

# Use the next line to set figure height and width (experiment to check the scale):
figure_width, figure_height = 7,7

# These lines set the figure title and axis labels and the font sizes:
fig_title = 'Figure Title'
x_label   = 'ULEV amounts per 100000 population'
y_label   = 'Charging device amounts per 100000 population'
title_fontsize = 18
label_fontsize = 16

#x_min, x_max = 0.5*np.min(data_use_array[:,0]), 1.1*np.max(data_use_array[:,0])
#y_min, y_max = 0.5*np.min(data_use_array[:,1]), 1.1*np.max(data_use_array[:,1])
# This is a function that sets up each figure's x-limits and y-limits and axis labels.

def setup_figure():
    
   # plt.xlim([x_min, x_max])
   # plt.ylim([y_min, y_max])
    plt.xlabel(x_label,fontsize=label_fontsize)
    plt.ylabel(y_label,fontsize=label_fontsize)


In [ ]:
data_by_cluster

In [ ]:
# This is a list of colours to differentiate each cluster.
color_list = ['b','r','g','m','c','k','y']
#Might be nice to have those on the same plot though.
# FIGURE N + 1 : COMBINED CLUSTER PLOT

# These lines create a plot with all the data points, coloured by cluster.
plt.figure(k_pref + 1,figsize=(figure_width,figure_height))
setup_figure()
plt.title(fig_title + ' - Coloured by Cluster',fontsize=title_fontsize)

for i in range(k_pref):
    
    x_values = data_by_cluster[i][:,0]
    y_values = data_by_cluster[i][:,1]
    
    plt.plot(x_values,y_values,color_list[i % 5] + '.')
    

SILHOUETTE score plot

In [ ]:
# calculate the SILHOUETTE SCORE of the cluster above (cluster number ==5)
import sklearn.metrics as sklm  # For the silhouette score
# These lines calculate the silhouette score...
silhouette_kmeans = sklm.silhouette_score(kmeans_data,kmeans.labels_)

# ... and print it:
print("Silhouette Score:", silhouette_kmeans)

In [ ]:
# cluster_numbers = range(1,10)
# a = list(cluster_numbers)

In [ ]:
all_scores = {'n':[],'ss':[]}
for n in range(2,10):
    # This line performs the k-means clustering:
    kmeans1 = sklc.KMeans(n_clusters=n, n_init=10,random_state=42)
    
    kmeans1_output = kmeans1.fit(kmeans_data)
    
    # This line creates a list giving the final cluster number of each point:
    clustering_ids_kmeans = kmeans1_output.labels_
    
    ss = sklm.silhouette_score(kmeans_data,clustering_ids_kmeans)
    
    all_scores['n'].append(n)
    all_scores['ss'].append(ss)
    

In [ ]:
Kmeans_result_df = pd.DataFrame(all_scores)

In [ ]:
plt.plot(Kmeans_result_df.n,Kmeans_result_df.ss)

plt.xlabel("Number of clusters")
plt.ylabel("Silhouette Score")

From the plot above can see cluster number of 5 is the best


In [ ]:
kmeans_data.info()

In [ ]:
cluster_df = pd.DataFrame(kmeans.labels_, index=kmeans_data.index)

In [ ]:
cluster_df

In [ ]:
# add back
ulev_cd_local_authority1_gdf_cluster = pd.merge(ulev_cd_local_authority1_gdf,cluster_df,on='LA_code',how='outer')

In [ ]:
ulev_cd_local_authority1_gdf_cluster.info()

check and drop NAs

In [ ]:
ulev_cd_local_authority1_gdf_cluster[ulev_cd_local_authority1_gdf_cluster[0].isna()]


In [ ]:
ulev_cd_local_authority1_gdf_cluster =ulev_cd_local_authority1_gdf_cluster.drop(index=19)

In [ ]:
ulev_cd_local_authority1_gdf_cluster =ulev_cd_local_authority1_gdf_cluster.drop(index=281)

In [ ]:
ulev_cd_local_authority1_gdf_cluster.shape

In [ ]:
ulev_cd_local_authority1_gdf_cluster.head(1)

Rename columns

In [ ]:
ulev_cd_local_authority1_gdf_cluster = ulev_cd_local_authority1_gdf_cluster.rename(columns={0:'cluster','per 100,000 population':'charging_devices_per_100000_population'})

In [ ]:
cluster_group = ulev_cd_local_authority1_gdf_cluster.groupby(['cluster'])['ULEV_per_100000_population','charging_devices_per_100000_population'].agg(['mean', 'count'])#agg(median_price='median').reset_index() # msoa listings grouped price
cluster_group

Map 

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,9))

ulev_cd_local_authority1_gdf_cluster.plot(ax=ax,column='cluster',legend=True)
#region1.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1,label='region',legend=True)
#ax.set_title('Countryside listings distribution', fontdict={'fontsize':'20', 'fontweight':'3'})  #provide a title

ax.axis('off')
#local_authority.plot(ax=ax,label='local_authority',edgecolor='black',facecolor='none',legend=True)
#region1.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1,label='region',legend=True)

### Linear regression


#### read in earning data

In [ ]:

earning = pd.read_excel (r'data.xlsx', sheet_name='earning')

In [ ]:
#earning = earning.rename(columns={'Unnamed: 0':'region'})

In [ ]:
earning

In [ ]:
earning = earning.drop(index=[0,1,13])

In [ ]:
earning = earning.rename(columns={'Region':'region'})

In [ ]:
earning.region[7]='East of England'

In [ ]:
earning = earning[['region',2019]]

In [ ]:
earning = earning.rename(columns={2019:'earning_2019'})

In [ ]:
earning

#### read in median house price data

In [ ]:
cols_house_price = []
for i in range(0,6):
    cols_house_price.append(i)
print(cols_house_price)

In [ ]:

median_house_price = pd.read_excel (r'data.xlsx', sheet_name='median_house_price',skiprows=4,usecols=cols_house_price)

In [ ]:
median_house_price['mean_median_house_price_2019'] = median_house_price.mean(axis=1)

In [ ]:
median_house_price = median_house_price[['Code','Name','mean_median_house_price_2019']].drop(index=[0,1])

In [ ]:
median_house_price = median_house_price.rename(columns={'Name':'region'})

In [ ]:
median_house_price.region.values

#### read in carbon_emission data

In [ ]:
cols_carbon_emission = [0,29]


In [ ]:
carbon_emission = pd.read_excel (r'data.xlsx', sheet_name='carbon_emission_direct_scope',skiprows=1,usecols=cols_carbon_emission)

In [ ]:
carbon_emission

In [ ]:
carbon_emission = carbon_emission.rename(columns={'Region/Country':'region','Per Capita Emissions (t)':'emissions_per_person_t_2019'})

In [ ]:
carbon_emission = carbon_emission.drop(index=11)

In [ ]:
carbon_emission['region'] = carbon_emission['region'].str.replace(' Total','')

In [ ]:
carbon_emission.region[2] = 'Yorkshire and The Humber'

In [ ]:
carbon_emission

#### read in education level data

In [ ]:
education = pd.read_excel (r'data.xlsx', sheet_name='education_level_2019end') 

In [ ]:
education = education.drop(index=[0,2,13])

In [ ]:
education

In [ ]:
education = education.drop(columns=['Unnamed: 1','Unnamed: 2'])

In [ ]:
education['region'][5]='Yorkshire and The Humber'

In [ ]:
education['region'][8]='East of England'

In [ ]:
education.region.values

#### read in rapid charging devices data

In [ ]:
rapid_charging = pd.read_excel (r'data.xlsx', sheet_name='rapid_charging_region') 

In [ ]:
rapid_charging = rapid_charging.rename(columns={'per 100,000 population':'rapid_charging_100000_pop','Name':'region'})

#### read in total charging devices data

In [ ]:
total_charging = pd.read_excel (r'data.xlsx', sheet_name='charging_device_region') 

In [ ]:
total_charging = total_charging[['region','charging_per_100000_pop']]

In [ ]:
total_charging

#### cars_registered

In [ ]:
cars_registered_row = pd.read_excel (r'data.xlsx', sheet_name='Car_new_regis_veh0254',skiprows=3) 

In [ ]:
cars_registered_row

In [ ]:
cars_registered_row = cars_registered_row.iloc[0:21]

In [ ]:
cars_registered_row

In [ ]:
cars_registered_row = cars_registered_row.drop(index=0)

In [ ]:
cars_registered_row

In [ ]:
cars_registered = cars_registered_row.transpose()

In [ ]:
cars_registered

In [ ]:
cars_registered['region']=cars_registered.index

In [ ]:
cars_registered.index = range(1, 18, 1)

In [ ]:
cars_registered

In [ ]:
cars_registered = cars_registered[[19,'region']]


In [ ]:
cars_registered_region = cars_registered.drop(index=[1,13,14,15,16,17])

In [ ]:
cars_registered_region = cars_registered_region.rename(columns={19:"cars_registered_thousands"})

In [ ]:
cars_registered_region.region[7]='East of England'

In [ ]:
cars_registered_region

#### ULEV registered 

In [ ]:
ulev_registered_raw = pd.read_excel (r'data.xlsx', sheet_name='ULEV_new_regis_veh0172 ',skiprows=3) 

In [ ]:
ulev_registered = ulev_registered_raw.iloc[47:51,0:12]

In [ ]:
ulev_registered

In [ ]:
ulev_registered.loc['Total']= ulev_registered.sum()

In [ ]:
ulev_registered_total = ulev_registered.loc[['Total']]

In [ ]:
ulev_registered_total

In [ ]:
ulev_registered_total = ulev_registered_total.transpose()

In [ ]:
ulev_registered_total

In [ ]:
ulev_registered_total = ulev_registered_total.drop(index='Year')

In [ ]:
ulev_registered_total['region'] = ulev_registered_total.index

In [ ]:
ulev_registered_region = ulev_registered_total.rename(columns={'Total':'ulevs_registered'})

In [ ]:
ulev_registered_region.region['East']='East of England'

In [ ]:
ulev_registered_region

#### merge the car_registered and the ulev_registered to calculate the market share

In [ ]:
ulev_registered_region

In [ ]:
cars_registered_region

In [ ]:
ulev_car_registered = ulev_registered_region.merge(cars_registered_region, on='region')

In [ ]:
ulev_car_registered

In [ ]:
ulev_car_registered['ulev_market_share_%'] = ulev_car_registered['ulevs_registered']/ulev_car_registered['cars_registered_thousands']/1000*100

In [ ]:
ulev_car_registered

In [ ]:
ulev_car_registered.region

In [ ]:
education.region

#### merge all the x data

In [ ]:
ulev_car_registered_house = pd.merge(ulev_car_registered,median_house_price, on='region',how='outer')

In [ ]:
ulev_car_registered_house

In [ ]:
ulev_car_registered_house_earning = pd.merge(ulev_car_registered_house,earning,on='region',how='outer')

In [ ]:
ulev_car_registered_house_earning

In [ ]:
ulev_car_registered_house_earning_carbonemission = pd.merge(ulev_car_registered_house_earning,carbon_emission,on='region',how='outer')

In [ ]:
ulev_car_registered_house_earning_carbonemission

In [ ]:
ulev_car_registered_house_earning_carbonemission_education = pd.merge(ulev_car_registered_house_earning_carbonemission,education,on='region',how='outer')

In [ ]:
ulev_car_registered_house_earning_carbonemission_education

In [ ]:
ulev_car_registered_house_earning_carbonemission_education_rapidchagring =  pd.merge(ulev_car_registered_house_earning_carbonemission_education,rapid_charging,on='region',how='outer')

In [ ]:
ulev_car_registered_house_earning_carbonemission_education_rapidchagring

In [ ]:
ulev_car_registered_house_earning_carbonemission_education_rapidchagring_totalcharging = pd.merge(ulev_car_registered_house_earning_carbonemission_education_rapidchagring,total_charging,on='region',how='outer')

In [ ]:
ulev_car_registered_house_earning_carbonemission_education_rapidchagring_totalcharging.columns

In [ ]:
regression_data = ulev_car_registered_house_earning_carbonemission_education_rapidchagring_totalcharging[['region','Code_x','ulev_market_share_%','mean_median_house_price_2019','earning_2019','emissions_per_person_t_2019','Qualified to NQF level 3 or above ','charging_per_100000_pop','rapid_charging_100000_pop']]

In [ ]:
regression_data = regression_data.rename(columns={'Qualified to NQF level 3 or above ':'Qualified to NQF level 3 or above'})

In [ ]:
regression_data

In [ ]:
regression_data["ulev_market_share_%"] = regression_data["ulev_market_share_%"].astype('float')
regression_data["charging_per_100000_pop"] = regression_data["charging_per_100000_pop"].astype('float')
regression_data["rapid_charging_100000_pop"] = regression_data["rapid_charging_100000_pop"].astype('float')

In [ ]:
regression_data.info()

In [ ]:
list1 = regression_data.columns.values[3:]

####  Linear regression

In [ ]:
plt.hist(regression_data['ulev_market_share_%'],bins=15)

In [ ]:
# first use the original independent variables

Y = regression_data['ulev_market_share_%']
X = regression_data.loc[:,list1]

In [ ]:
plt.rcParams.update({'font.size': 14})

fig2, axes2 = plt.subplots(2,3, figsize=(15,6))
#axes[1][2].set_visible(False)
#plt.style.use('_mpl-gallery')
#axes[1][0].set_position([0.24,0.125,0.228,0.343])
#axes[1][1].set_position([0.55,0.125,0.228,0.343])


#fig2.tight_layout(pad=5)

for i in range(2):
    for j in range(3):
        #if (i==1 and j==2):
            #continue
        axes2[i,j].hist(regression_data[list1[j+3*i]],bins=10,edgecolor="white",color='xkcd:salmon')
        #axes[i,j].set_title(list1[j+3*i])
        axes2[i,j].set(xlabel= list1[j+3*i])
        
#fig2.suptitle('Distribution of independent variables',fontsize=20)

In [ ]:
plt.rcParams.update({'font.size': 13})

fig3, axes3 = plt.subplots(2,3, figsize=(15,6))
#axes[1][2].set_visible(False)

#axes[1][0].set_position([0.24,0.125,0.228,0.343])
#axes[1][1].set_position([0.55,0.125,0.228,0.343])


fig3.tight_layout(pad=10)

for i in range(2):
    for j in range(3):
        #if (i==1 and j==2):
            #continue
        axes3[i,j].scatter(regression_data[list1[j+3*i]], regression_data['ulev_market_share_%'],color='xkcd:salmon')
        #axes[i,j].set_title(list1[j+3*i])
        axes3[i,j].plot(np.unique(regression_data[list1[j+3*i]]), np.poly1d(np.polyfit(regression_data[list1[j+3*i]], regression_data['ulev_market_share_%'], 1))(np.unique(regression_data[list1[j+3*i]])),color='blue',linestyle='dashed')
        axes3[i,j].set(xlabel= list1[j+3*i])
        axes3[i,j].set(ylabel= 'ULEV market share')
        
#fig3.suptitle('scatter plot',fontsize=20)

VIF 

In [ ]:


df = X
plt.rcParams["axes.grid"] = False
f = plt.figure(figsize=(12, 8))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=90)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In [ ]:

# This function is adjusted from: https://stackoverflow.com/a/51329496/4667568
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, thresh=5):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: (default 5) the threshould VIF value. If the VIF of a variable is greater than thresh, it should be removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constatnt item to the data. add_constant is a function from statsmodels (see the import above)
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const
        vif_df = vif_df.drop('const')
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        # If there are multiple variabels with VIF>thresh, only one of them is removed. This is because we want to keep as many variables as possible
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

In [ ]:
X

In [ ]:
# do the VIF
X_dropped = drop_column_using_vif_(X)

In [ ]:
# now do the regression with VIF since the model_0's issue
# first do a correlation matrix for X

df = X_dropped
plt.rcParams["axes.grid"] = False
f = plt.figure(figsize=(12, 8))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=90)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In [ ]:
X1 = X_dropped
X1 = sm.add_constant(X1)

model_1=sm.OLS(Y,X1).fit()
model_1.summary()

In [ ]:
# for residual plot 
plt.figure(figsize=(6,2))
plt.rcParams.update({'font.size': 8})
plt.scatter(model_1.fittedvalues, model_1.resid)
# adding title and labels
plt.xlabel('Fitted value')
plt.ylabel('Residual')
plt.title('Model1_Residual vs. Fitted')
plt.show()



remove two variables with large P-values and do regression Model_2

In [ ]:
X2 = regression_data[['emissions_per_person_t_2019','charging_per_100000_pop']]
X2 = sm.add_constant(X2)

model_2=sm.OLS(Y,X2).fit()
model_2.summary()

In [ ]:
# for residual plot 
plt.figure(figsize=(6,2))
plt.rcParams.update({'font.size': 8})
plt.scatter(model_2.fittedvalues, model_2.resid)
# adding title and labels
plt.xlabel('Fitted value')
plt.ylabel('Residual')
plt.title('Model2_Residual vs. Fitted')
plt.show()

